In [1]:
import csv
import torch
import numpy as np
import pandas as pd

fields = ['input1', 'input2','output1', 'output2']
rows = []
h_target = torch.tensor([[1.,1.],[1.,-1]] / np.sqrt(2), dtype=torch.complex128)

for i in range(8):
    phi = np.random.rand() * 2*np.pi
    theta = np.random.rand() * np.pi
    
    input = torch.tensor([np.sin(theta), np.cos(theta) * np.exp(1j * phi)])
    output = h_target @ input

    rows.append([np.sin(theta), np.cos(theta) * np.exp(1j * phi), output[0].item(), output[1].item()])

filename = 'qubit.csv'

with open(filename, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)
    csvwriter.writerows(rows)

In [5]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset

num_qubits = 8

num_epochs = 100

learning_rate = 0.01

class input_vec_dataset(Dataset):
    def __init__(self):
        
        self.df = pd.read_csv('qubit.csv')
        self.df['input1'] = self.df['input1'].astype(complex)
        self.df['input2'] = self.df['input2'].astype(complex)
        self.df['output1'] = self.df['output1'].astype(complex)
        self.df['output2'] = self.df['output2'].astype(complex)

        dataset = []
        labels = []

        for i in range(num_qubits):
            dataset.append([self.df['input1'][i],self.df['input2'][i]])
            
        self.dataset = torch.tensor(dataset, dtype=torch.complex128)
        
        for i in range(num_qubits):
            labels.append([self.df['output1'][i],self.df['output2'][i]])
        
        self.labels = torch.tensor(labels, dtype=torch.complex128)
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        return self.dataset[idx],self.labels[idx]

data_set = input_vec_dataset()

data_loader = torch.utils.data.DataLoader(data_set, batch_size=1, shuffle=True, drop_last=False)

class HModel(torch.nn.Module):
    
    def __init__(self):
        super(HModel, self).__init__()     
        self.linear = nn.Linear(2,2,bias=False, dtype=torch.complex128)
        
    def forward(self, x):
        x = self.linear(x)
        return x
    
model = HModel()

class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, inputs, targets):
        loss = 1 - (abs(torch.dot(torch.conj(inputs).t().reshape(-1), targets.reshape(-1))))**2
        loss.requires_grad = True
        return loss.mean()
    
loss_fn = CustomLoss()
# criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    
    for batch, labels in data_loader:
 
        optimizer.zero_grad()
        
        outputs = model(batch)
        
        loss = loss_fn(batch,labels)
        
        loss.backward()
        
        optimizer.step()
        
    if epoch % 10 == 0:
        
        print(f'epoch: {epoch}, loss: {loss}')
        
model.eval()

with torch.no_grad():

    out_data = model(torch.tensor([0+0j,1+0j],dtype=torch.complex128))
    print(out_data)
#     print(np.conj(out_data) * out_data) 
    print(torch.abs(out_data))

epoch: 0, loss: 0.9731467346058638
epoch: 10, loss: 0.9682599351397435
epoch: 20, loss: 0.7854467185241013
epoch: 30, loss: 0.9634618561443786
epoch: 40, loss: 0.9731467346058638
epoch: 50, loss: 0.7854467185241013
epoch: 60, loss: 0.7854467185241013
epoch: 70, loss: 0.9731467346058638
epoch: 80, loss: 0.9986281832383886
epoch: 90, loss: 0.9682599351397435
tensor([-0.2538+0.5125j,  0.6959+0.4637j], dtype=torch.complex128)
tensor([0.5719, 0.8362], dtype=torch.float64)


In [6]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset

num_qubits = 8

num_epochs = 100

learning_rate = 0.01

class input_vec_dataset(Dataset):
    def __init__(self):
        
        self.df = pd.read_csv('qubit.csv')
        self.df['input1'] = self.df['input1'].astype(complex)
        self.df['input2'] = self.df['input2'].astype(complex)
        self.df['output1'] = self.df['output1'].astype(complex)
        self.df['output2'] = self.df['output2'].astype(complex)

        dataset = []
        labels = []

        for i in range(num_qubits):
            dataset.append([self.df['input1'][i],self.df['input2'][i]])
            
        self.dataset = torch.tensor(dataset, dtype=torch.complex128)
        
        for i in range(num_qubits):
            labels.append([self.df['output1'][i],self.df['output2'][i]])
        
        self.labels = torch.tensor(labels, dtype=torch.complex128)
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        return self.dataset[idx],self.labels[idx]

data_set = input_vec_dataset()

data_loader = torch.utils.data.DataLoader(data_set, batch_size=1, shuffle=True, drop_last=False)

class HModel(torch.nn.Module):
    
    def __init__(self):
        super(HModel, self).__init__()     
        self.linear = nn.Linear(2,2,bias=False, dtype=torch.complex128)
        
    def forward(self, x):
        x = self.linear(x)
        return x
    
model = HModel()

class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, inputs, targets):
        loss = 1 - (abs(torch.dot(torch.conj(inputs).t().reshape(-1), targets.reshape(-1))))**2
        return loss.mean()
    
loss_fn = CustomLoss()
# criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    
    for batch, labels in data_loader:
 
        optimizer.zero_grad()
        
        outputs = model(batch)
        
        loss = loss_fn(outputs,labels)
        
        loss.backward()
        
        optimizer.step()
        
    if epoch % 10 == 0:
        
        print(f'epoch: {epoch}, loss: {loss}')
        
model.eval()

with torch.no_grad():

    out_data = model(torch.tensor([0+0j,1+0j],dtype=torch.complex128))
    print(out_data)
#     print(np.conj(out_data) * out_data) 
    print(torch.abs(out_data))

epoch: 0, loss: 0.30275406215435274
epoch: 10, loss: -3.7769435264595517
epoch: 20, loss: -10.843307128995399
epoch: 30, loss: -27.369222249447283
epoch: 40, loss: -52.0466540101741
epoch: 50, loss: -69.83441154375755
epoch: 60, loss: -108.52018785109139
epoch: 70, loss: -171.9502337789074
epoch: 80, loss: -192.27735058367554
epoch: 90, loss: -230.2121636379882
tensor([-8.7354+8.6468j, 10.5804-10.7509j], dtype=torch.complex128)
tensor([12.2913, 15.0840], dtype=torch.float64)
